In [ ]:
import numpy as np
from numpy.random import default_rng
import csv

class Neural_Network:
    def __init__(self, definition_array, activation_function, batch_size=32):
        self.definition = definition_array
        self.activation_function = activation_function
        self.batch_size = batch_size
        self.layers = []

        for i in range(len(definition_array)):
            prev_layer = self.layers[i - 1] if i > 0 else None
            prev_neuron_count = self.layers[i - 1].Neuron_count if i > 0 else 0
            self.layers.append(Layer(prev_layer, definition_array[i], prev_neuron_count,
                                     activation_function, batch_size))

    def feedforward(self, input_batch):
        self.layers[0].activation_values = input_batch
        for layer in self.layers[1:]:
            layer.feedforward()
        self.output_values = softmax(self.layers[-1].activation_values)

    def backpropagate(self, target_batch):
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            if i == len(self.layers) - 1:
                output_error = layer.activation_values - target_batch
                layer.delta = output_error * deriv_sigmoid(layer.activation_values)
            else:
                next_layer = self.layers[i + 1]
                layer.delta = np.dot(next_layer.weights.T, next_layer.delta) * deriv_sigmoid(layer.activation_values)
            if layer.prev_Layer is not None:
                layer.weights -= np.dot(layer.delta, layer.inputs.T) * 0.005 / self.batch_size
                layer.biases -= np.mean(layer.delta, axis=1, keepdims=True) * 0.5
    
    def train(self, datafile_path, epochs):
        for epoch in range(epochs):
            with open(datafile_path, mode='r') as file:
                datafile = csv.reader(file)
                batch_inputs = []
                batch_targets = []
                correct_predictions = 0
                for i, line in enumerate(datafile):
                    target = np.zeros(10)
                    target[int(line[0])] = 1.0
                    inputs = np.array([float(line[i+1]) for i in range(784)])
                    batch_inputs.append(inputs)
                    batch_targets.append(target)
                    if (i + 1) % self.batch_size == 0:
                        batch_inputs = np.array(batch_inputs).T
                        batch_targets = np.array(batch_targets).T
                        self.feedforward(batch_inputs)
                        correct_predictions += np.sum(np.argmax(self.output_values, axis=0) == np.argmax(batch_targets, axis=0))
                        self.backpropagate(batch_targets)
                        batch_inputs, batch_targets = [], []

                print(f"Epoch {epoch + 1}/{epochs}, Correct Predictions: {correct_predictions}, Accuracy: {correct_predictions/600:.2f}%")

    def test(self, datafile_path):
        with open(datafile_path, mode='r') as file:
            datafile = csv.reader(file)
            correct_predictions = 0
            total_samples = 10000
            for i, line in enumerate(datafile):
                target = np.zeros(10)
                target[int(line[0])] = 1.0
                inputs = np.array([float(line[row * 28 + col + 1]) for row in range(28) for col in range(28)])
                self.feedforward(inputs[:, np.newaxis])
                if np.argmax(self.output_values) == int(line[0]):
                    correct_predictions += 1
                if (i + 1) == total_samples:
                    break
            accuracy = (correct_predictions / total_samples) * 100
            print(f"Test Accuracy: {accuracy:.2f}%")

class Layer:
    def __init__(self, prev_Layer, Neuron_count, prev_Layer_Neuron_count, activation_function, batch_size):
        self.prev_Layer = prev_Layer
        self.Neuron_count = Neuron_count
        self.activation_function = activation_function
        self.batch_size = batch_size
        self.inputs = None if prev_Layer is None else np.zeros((prev_Layer_Neuron_count, batch_size))
        self.weights = default_rng(42).random((Neuron_count, prev_Layer_Neuron_count)) if prev_Layer else None
        self.biases = np.zeros((Neuron_count, 1))
        self.activation_values = np.zeros((Neuron_count, batch_size))
        self.delta = np.zeros((Neuron_count, batch_size))

    def feedforward(self):
        if self.prev_Layer is None:
            return
        self.inputs = self.prev_Layer.activation_values
        z = np.dot(self.weights, self.inputs) + self.biases
        if self.activation_function == "sigmoid":
            self.activation_values = sigmoid(z)
        elif self.activation_function == "relu":
            self.activation_values = relu(z)
        elif self.activation_function == "tanh":
            self.activation_values = tanh(z)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)

def deriv_sigmoid(x):
    sx = sigmoid(x)
    return sx * (1 - sx)



In [ ]:
NeN = Neural_Network([784, 400, 10], "sigmoid", batch_size=32)
NeN.train('mnist_train.csv', epochs=40)
NeN.test('mnist_test.csv')

In [17]:
import csv
import numpy as np
for i in range(5):
    txt = open("save" + str(i+1) + ".txt")
    textlength = len(txt.readlines())
    labelarray = txt.readline(textlength)
    labelarray = np.array(txt.readline(textlength).split(", "))
    print(txt.readlines(), "save" + str(i+1) + ".txt")
    for v in range(textlength-1):
        line = txt.readline(v+1)
        linearray = np.array(line.split(" "))
        print(linearray, i, v+1)
     ##   linearray = linearray.astype(np.int32)
        

[] save1.txt
[''] 0 1
[''] 0 2
[''] 0 3
[''] 0 4
[''] 0 5
[''] 0 6
[''] 0 7
[''] 0 8
[''] 0 9
[''] 0 10
[''] 0 11
[''] 0 12
[''] 0 13
[''] 0 14
[''] 0 15
[''] 0 16
[''] 0 17
[''] 0 18
[] save2.txt
[''] 1 1
[] save3.txt
[''] 2 1
[] save4.txt
[''] 3 1
[''] 3 2
[''] 3 3
[''] 3 4
[''] 3 5
[''] 3 6
[''] 3 7
[''] 3 8
[''] 3 9
[''] 3 10
[''] 3 11
[''] 3 12
[''] 3 13
[''] 3 14
[''] 3 15
[''] 3 16
[''] 3 17
[''] 3 18
[''] 3 19
[''] 3 20
[''] 3 21
[''] 3 22
[''] 3 23
[''] 3 24
[''] 3 25
[''] 3 26
[] save5.txt
[''] 4 1
[''] 4 2
[''] 4 3
[''] 4 4
[''] 4 5
[''] 4 6
[''] 4 7
[''] 4 8
[''] 4 9
[''] 4 10
[''] 4 11
[''] 4 12
[''] 4 13
[''] 4 14
[''] 4 15
[''] 4 16
[''] 4 17
[''] 4 18
[''] 4 19
[''] 4 20
[''] 4 21
[''] 4 22
[''] 4 23
[''] 4 24
[''] 4 25
[''] 4 26
[''] 4 27
[''] 4 28
[''] 4 29
[''] 4 30
[''] 4 31
[''] 4 32
[''] 4 33
[''] 4 34
[''] 4 35
[''] 4 36
[''] 4 37
[''] 4 38
[''] 4 39
[''] 4 40
[''] 4 41
[''] 4 42
[''] 4 43
[''] 4 44
